In [ ]:
using Random
using Distributions
using Statistics
using Plots
using LinearAlgebra
using HypothesisTests
using QuadGK

# Example
Estimate
$$
\int_0^1 \cos(x/5) e^{-5x}dx=\mathbb{E}[\cos(U/5)e^{-5U}], \quad U \sim U(0,1)
$$
by Monte Carlo

In [ ]:
I = quadgk(x -> cos(x/5) * exp(-5*x), 0, 1)[1]

In [ ]:
f = x-> cos(x/5) * exp(-5*x)

xx = 0:0.01:1
plot(xx, f.(xx), label="cos(x/5)exp(-5x)", lw=2)
xlabel!("x")

## Naive Estimate
Use $U(0,1)$ r.v.'s.

In [ ]:
n_samples = 10^2;

Random.seed!(100)
U_samples = rand(n_samples);
f_samples = f.(U_samples);

Ef_est = mean(f_samples)
println("Estimated integral = $Ef_est")
println("Variance = $(var(f_samples))")

α = 0.05;
z=quantile(Normal(), 1-α/2);

Ztest = OneSampleZTest(mean(f_samples), std(f_samples), n_samples);
println("95%% CI = ($(Ztest.xbar-z*Ztest.stderr), $(Ztest.xbar+z*Ztest.stderr))");

## Importance Sampling
Use distribution with pdf
$$
f_Y(y) = \frac{e^{-5y}1_{[0,1]}(y)}{c}
$$
to write
$$
\mathbb{E}[\cos(X/5)e^{-5X}] = c\mathbb{E}[\cos(Y/5)].
$$
The normalization is 
$$
c = (1-e^{-5})/5.
$$
Observe that the $\cos(x/5)$ is a very slowly varying function over $[0,1]$:

In [ ]:
xx = 0:0.01:1
plot(xx, cos.(xx/5), label="cos(y/5)", lw=2)
xlabel!("y")

To sample more complex distributions, use the `Distributions.jl`.  This supports the $U(a,b)$ distribution,
```
a= 1; b = 5;
unif_dist = Uniform(a,b);
```
which can then be sampled as
```
rand(unif_dist)
```
or
```
samples = rand(unif_dist, 100);
```

Analogously, the exponential distribution  with rate $1/\lambda$  can be sampled as
```
exponential_dist = Exponential(1/λ);
```
Truncated distributions can then be constructed from a given distribution, `dist`, as
```
trunc_dist = truncated(dist, a,b);
```
where `a` and `b` are the lower and upper limits fo the region from which we wish to truncate to.

In [ ]:
exponential_dist = Exponential(1/5.0); # construct the Exponential distribution with rate 1/5.0
trunc_dist = truncated(exponential_dist, 0.0, 1.0); # truncate to [0,1] 

In [ ]:
Z_samples = rand(exponential_dist, 10^6);
histogram(Z_samples, bins=30, density=true, label="Exp(5)", norm=:pdf)
zz = LinRange(0, maximum(Z_samples), 100)
plot!(zz, 5.0 * exp.(-5.0 * zz), lw=2, label="Exp(5)")
xlabel!("z")
ylabel!("Density")  

In [ ]:
Y_samples = rand(trunc_dist, 10^6);
histogram(Y_samples, bins=30, density=true, label="Truncated Exp(5)", norm=:pdf)
xlabel!("y")
ylabel!("Density")  

In [ ]:

Random.seed!(100)
y_samples = rand(trunc_dist, n_samples);
c = (1 - exp(-5)) / 5.0;
g_samples = cos.(y_samples/5);

Ef_est = c * mean(g_samples)
println("Estimated integral = $Ef_est")
println("Variance = $(var(g_samples))")

α = 0.05;
z=quantile(Normal(), 1-α/2);

Ztest = OneSampleZTest(mean(c* g_samples), std(c* g_samples), n_samples);
println("95%% CI = ($(Ztest.xbar-z*Ztest.stderr), $(Ztest.xbar+z*Ztest.stderr))");